#### To play around with interactive widgets, *Fork* the notebook!

In [ ]:
import numpy as np 
import pandas as pd 

from path import Path
import os
from functools import partial

from ipywidgets import interact, interact_manual, fixed
from IPython.display import display

import cufflinks as cf
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from scipy.integrate import odeint, solve_ivp

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('use_inf_as_na', True)
cf.set_config_file(offline=True, theme='solar');

# SIR Model

This simple visualistion tells how the SIR model is assigning people to different groups

![SIR Model](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fsrep46076/MediaObjects/41598_2017_Article_BFsrep46076_Fig1_HTML.jpg)

## Increase in Infected People


$$ 
\begin{equation}
I_{t+1} = I_{t} + P_{contact} * P_{spread} * \frac{I_{t}}{N} * S_{t} - P_{recover} * I_{t}
\end{equation}
$$

Let's understand every component of the RHS


$$ 
\begin{equation}
I_{t+1} = \overbrace{I_{t}}^{I_{t}} + \overbrace{P_{contact} * P_{spread} * \frac{I_{t}}{N} * S_{t}}^{S_{t-1} - S_{t}} - \overbrace{P_{recover} * I_{t}}^{R_{t}}
\end{equation}
$$

where, 
* $I_{t}$ is the number of infected people at time $t$

* $N$ is the total population of the region. For a closed population SIR model $\forall{t}: N = S_{t} + I_{t} + R_{t}$  

* $S_{t}$ is the number of susceptible people at time $t$

* $P_{contact}$ is the probability that a suspectible person contacts an infected person

* $P_{spread}$ is the probability that a suspectible person gets infected (after contact)

* $P_{recover}$ is the probability of an infected person recovering (or dying)

Simplifying the equation by assuming: 

* $\beta = P_{spread}*P_{contact}$
* $\gamma = P_{recover}$

The equation reduces to :


$$
\begin{equation}
I_{t+1} = I_{t} + \beta*(\frac{I_{t}}{N}) * S_{t} - \gamma*I_{t} 
\end{equation}
$$

## Significance of $\beta$ and $\gamma$

*T.D.*

Furthermore,

1. Reproduction Rate: $R_{0} = \frac{\beta}{\gamma}$  
2. Vaccination Threshold: $V_{0} > 1 - \frac{1}{R_{0}}$ 

## Differential equation for $I_{t}$

$\implies I_{t+1} - I_{t} = \beta*(\frac{I_{t}}{N}) * S_{t} - \gamma*I_{t} $

$\implies I_{t+\Delta t} - I_{t} = \beta*\Delta t*(\frac{I_{t}}{N}) * S_{t} - \gamma*\Delta t*I_{t} $

Dividing both sides by $\Delta t $:

$\implies \frac{I_{t+1} - I_{t}}{\Delta t} = \frac{\beta*\Delta t*(\frac{I_{t}}{N}) * S_{t} - \gamma*\Delta t*I_{t}}{\Delta t} $

$\implies \frac{dI_{t}}{dt} = \beta*(\frac{I_{t}}{N}) * S_{t} - \gamma*I_{t} $

$\implies I_{t}' = \beta*(\frac{I_{t}}{N}) * S_{t} - \gamma*I_{t} $



In [ ]:
def solve_I(β, ɣ, N, I, S):

    dI_dt = β*(I/N)*S - ɣ*I
    return dI_dt

## Differential equation for $S_{t}$

After every unit of time passes, some susceptible people go from $S \longrightarrow I$, hence reducing $S$' population. This decrement is given by,

$ S_{t+1} = S_{t} - \beta*(\frac{I_{t}}{N}) * S_{t}$

$ \implies S_{t+1} - S_{t} =  - \beta*(\frac{I_{t}}{N}) * S_{t}$


Dividing both sides by $\Delta t = (t + 1) - t = 1 $:

$\implies \frac{dS_{t}}{dt} =  - \beta*(\frac{I_{t}}{N}) * S_{t} $


In [ ]:
def solve_S(β, N, I, S):

    dS_dt = - β*(I/N)*S
    return dS_dt

## Differential equation for $R_{t}$

After every unit of time passes, some infected people go from $I \longrightarrow R$, hence increasing $R$'s population. This increment is given by,

$ R_{t+1} = R_{t} + \gamma * I_{t} $

$\implies R_{t+1} - R_{t} = \gamma * I_{t} $

Dividing both sides by $\Delta t = (t + 1) - t = 1 $:

$\implies \frac{dR_{t}}{dt} = \gamma * I_{t}  $



In [ ]:
def solve_R(ɣ, I):
    dR_dt = ɣ*I
    return dR_dt

# Complete Model

In [ ]:
def sir_model(β, ɣ, N, t, Y):
    
    I, S, R = Y
    dS_dt = - β*(I/N)*S
    dR_dt = ɣ*I
    dI_dt = - dS_dt - dR_dt
    
    return [dI_dt, dS_dt, dR_dt]

In [ ]:
@ interact(β=(0.0, 1., 0.01), ɣ=(0.01, 1., 0.01))
def sir_play(β=0.15, ɣ=0.15):
    
    
    N = 10_000
    I0 = 10
    R0 = 0
    S0 = N - I0 - R0
    
    t = np.linspace(0, 90, 180)
    
#     I = odeint(func=partial(sir_model, β, ɣ, N), y0=[I0, S0, R0], t=t, tfirst=True)
    I = solve_ivp(fun=partial(sir_model, β, ɣ, N), y0=[I0, S0, R0], t_eval=t, t_span=(0, len(t))).y.T

    print(f"Reproduction Rate is {β/ɣ}")
    pd.DataFrame(index=t, data=I, columns=['I', 'S', 'R']).iplot(y=['I', 'R'], secondary_y=['S'])

# Things you can try:

1. The biggest drawback of SIR model is that a 'recovered' patient can either be healthy (alive after infection) or dead -_-.

    Try building SIR-D model, in which an infected person can move from infected to dead too

    *Hint*:
    Add a parameter $\delta$ to account for the death rate
    
    
2. Play with values of $\beta$, $\gamma$ and see what happens when reproduction rate is < 1 or = 1 or > 1. 